In [1]:
print("ajja")

ajja


In [2]:
import random

class Guess:
    def __init__(self, word):
        self.word = word
        self.current_guess = ['-' for _ in word]
        self.letters_guessed = set()

    def display_menu(self):
        print("1. Guess\n2. Tell me\n3. Letter\n4. Quit")

    def display_status(self):
        print("Current Guess:", ' '.join(self.current_guess))
        print("Letters Guessed:", ', '.join(sorted(self.letters_guessed)))

    def guess_word(self, guessed_word):
        if guessed_word.lower() == self.word.lower():
            return True
        return False

    def guess_letter(self, letter):
        if letter.lower() in self.letters_guessed:
            print(f"You already guessed the letter '{letter}'. Try again.")
        elif letter.lower() in self.word.lower():
            print(f"Good guess! '{letter}' is in the word.")
            self.update_current_guess(letter.lower())
            self.display_letter_frequency(letter.lower())  # Uncomment if needed
        else:
            print(f"Sorry, '{letter}' is not in the word.")
        self.letters_guessed.add(letter.lower())

    def display_letter_frequency(self, letter):
        frequency = self.calculate_letter_points(letter)
        print(f"Frequency of '{letter}': {frequency:.2%}")

    def update_current_guess(self, letter):
        for i in range(len(self.word)):
            if self.word[i].lower() == letter:
                self.current_guess[i] = letter

    def tell_me(self):
        print("You give up and ask the game to show you the correct word.")
        print(f"The correct word is: {self.word}")

    def calculate_score(self):
        total_letters = len(self.word)
        if total_letters == 0:
            return 0

        return sum([self.calculate_letter_points(letter) for letter in self.word]) / 4

    def display_final_report(self, score):
        print("Final Report:")
        print("{:<15} {:<15} {:<15} {:<15} {:<15}".format("Word", "Status", "Bad Guesses", "Missed Letters","Latest Result"))
        word = self.word
        current_guess = self.current_guess
        letters_guessed = self.letters_guessed
        final_status = "Correct" if '-' not in current_guess else "Give Up"
        bad_guesses =(len(letters_guessed) - current_guess.count('-'))-4
        missed_letters = current_guess.count('-')
        negativescore=bad_guesses*0.10
        scoree=score
        #print("SCOREEE",scoree)
        new_rsl=(scoree*100)-(negativescore)
        #print("SCOREEE",scoree)
        #print("Negative Score ",negativescore)
        # print("NEW SCORE: ",new_rsl)
        #f"{score:.2%}"
        print("{:<15} {:<15} {:<15} {:<15} {:<15}".format(word, final_status, bad_guesses, missed_letters, new_rsl))


    def display_letter_frequencies(self):
        letter_frequencies = self.calculate_letter_frequencies()
        print("Letter Frequencies:")
        for letter, frequency in letter_frequencies.items():
            print(f"{letter}: {frequency:.2%}")

    def calculate_letter_frequencies(self):
        letter_frequencies = {}
        for letter in self.word:
            if letter.isalpha():
                letter_frequencies[letter] = letter_frequencies.get(letter, 0) + 1

        total_occurrences = sum(letter_frequencies.values())
        letter_frequencies_percentage = {letter: count / total_occurrences for letter, count in letter_frequencies.items()}

        return letter_frequencies_percentage

    def calculate_letter_points(self, letter):
        letter_frequencies = {
            'a': 8.17, 'b': 1.49, 'c': 2.78, 'd': 4.25, 'e': 12.70,
            'f': 2.23, 'g': 2.02, 'h': 6.09, 'i': 6.97, 'j': 0.15,
            'k': 0.77, 'l': 4.03, 'm': 2.41, 'n': 6.75, 'o': 7.51,
            'p': 1.93, 'q': 0.10, 'r': 5.99, 's': 6.33, 't': 9.06,
            'u': 2.76, 'v': 0.98, 'w': 2.36, 'x': 0.15, 'y': 1.97,
            'z': 0.07
        }

        return letter_frequencies.get(letter.lower(), 0.0) / 100

    def play(self, test_mode):
        if test_mode:
            print("Test Mode: The correct word is:", self.word)
            print("Now you can continue with the game.")

        while '-' in self.current_guess:
            print("\033[2J\033[;H", end='')
            self.display_status()
            self.display_menu()
            choice = input("Enter your choice (1-4): ")

            if choice == '1':
                guessed_word = input("Enter your guess for the whole word: ")
                if self.guess_word(guessed_word):
                    print("Congratulations! You guessed the word.")
                    score = self.calculate_score()
                    print(f"Your score for this game is: {score:.2f}")
                    self.display_final_report(score)
                    return
                else:
                    print("Sorry, incorrect guess. Keep trying!")

            elif choice == '2':
                self.tell_me()
                score = self.calculate_score()
                print(f"Your score for this game is: {score:.2f}")
                self.display_final_report(score)
                return

            elif choice == '3':
                letter = input("Enter a letter to guess: ")
                self.guess_letter(letter)

            elif choice == '4':
                print("Thanks for playing! Goodbye.")
                return

            else:
                print("Invalid choice. Please choose again.")

        print("Congratulations! You guessed the word:", self.word)
        score = self.calculate_score()
        print("Your score for this game is ",score*100)
        self.display_final_report(score)
        return score

class Game:
    def __init__(self):
        self.games_played = []

    def play_game(self, test_mode):
        playing = True
        while playing:
            string_database = StringDatabase()
            word = string_database.get_random_word()
            guess = Guess(word)
            score = guess.play(test_mode)
            self.games_played.append((word, score))
            
            play_again = input("Play again (y/n)? ")
            if play_again.lower() != 'y':
                playing = False
                self.display_summary()

    def display_summary(self):
        print("\nFINAL REPORT")
        print("-" * 15)
        total_score = 0
        for word, score in self.games_played:
            score=score*100
            print(f"{word}: {score:.2f}")
            total_score += score
        print(f"\nTotal Score: {total_score:.2f}")

class StringDatabase:
    def __init__(self):
        self.words = self.load_words()

    def load_words(self):
        try:
            with open("four_letters.txt", "r") as file:
                words = file.read().split()
            return words
        except FileNotFoundError:
            print("Error: 'four_letters.txt' not found.")
            return []

    def get_random_word(self):
        return random.choice(self.words)

def main():
    mode_choice = input("Choose a mode (1 for Play mode, 2 for Test mode): ")

    if mode_choice == '1':
        test_mode = False
    elif mode_choice == '2':
        test_mode = True
    else:
        print("Invalid choice. Exiting.")
        return

    game = Game()
    game.play_game(test_mode)

if __name__ == "__main__":
    main()


Choose a mode (1 for Play mode, 2 for Test mode): 2
Test Mode: The correct word is: fact
Now you can continue with the game.
Current Guess: - - - -
Letters Guessed: 
1. Guess
2. Tell me
3. Letter
4. Quit
Enter your choice (1-4): 3
Enter a letter to guess: a
Good guess! 'a' is in the word.
Frequency of 'a': 8.17%
Current Guess: - a - -
Letters Guessed: a
1. Guess
2. Tell me
3. Letter
4. Quit
Enter your choice (1-4): 3
Enter a letter to guess: f
Good guess! 'f' is in the word.
Frequency of 'f': 2.23%
Current Guess: f a - -
Letters Guessed: a, f
1. Guess
2. Tell me
3. Letter
4. Quit
Enter your choice (1-4): 3
Enter a letter to guess: c
Good guess! 'c' is in the word.
Frequency of 'c': 2.78%
Current Guess: f a c -
Letters Guessed: a, c, f
1. Guess
2. Tell me
3. Letter
4. Quit
Enter your choice (1-4): 3
Enter a letter to guess: u
Sorry, 'u' is not in the word.
Current Guess: f a c -
Letters Guessed: a, c, f, u
1. Guess
2. Tell me
3. Letter
4. Quit
Enter your choice (1-4): 3
Enter a letter t